# **📘 Transformer-based Neural Machine Translation**

## **“Attention Is All You Need” – English → Spanish**

**1. Environment Setup & Version Check**

In [3]:
!pip install -U tensorflow==2.16.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.8/590.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 106.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Succes

In [2]:
!pip install -U ml_dtypes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 82.9 MB/s eta 0:00:00
  Attempting uninstall: ml_dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.12.0 requires tf_keras~=2.19, which is not installed.
dopamine-rl 4.1.2 requires tf-keras>=2.18.0, which is not installed.
tensorflow 2.16.2 requires ml-dtypes~=0.3.1, but you have ml-dtypes 0.5.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.16.2 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.1

In [15]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("GPU:", tf.config.list_physical_devices("GPU"))


TensorFlow: 2.19.0
Keras: 3.10.0
GPU: []


**2. Dataset Extraction (spa-eng)**

In [16]:
!unzip spa-eng.zip


Archive:  spa-eng.zip
replace spa-eng/_about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: spa-eng/_about.txt      
replace spa-eng/spa.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: spa-eng/spa.txt         


In [17]:
!ls
!ls spa-eng


sample_data  spa-eng  spa-eng.zip
_about.txt  spa.txt


**3. Load and Clean Dataset**

In [18]:
from pathlib import Path

text = Path("spa-eng/spa.txt").read_text(encoding="utf-8")
text = text.replace("¡", "").replace("¿", "")

print("Total characters:", len(text))
print(text.splitlines()[:5])


Total characters: 7901922
['Go.\tVe.', 'Go.\tVete.', 'Go.\tVaya.', 'Go.\tVáyase.', 'Hi.\tHola.']


**4. Create Sentence Pairs**

In [19]:
lines = text.splitlines()
pairs = [line.split("\t") for line in lines]
pairs = [(en, es) for en, es in pairs if en and es]

np.random.seed(42)
np.random.shuffle(pairs)

print("Total sentence pairs:", len(pairs))
print("Sample:", pairs[:3])


Total sentence pairs: 118964
Sample: [('How boring!', 'Qué aburrimiento!'), ('I love sports.', 'Adoro el deporte.'), ('Would you like to swap jobs?', 'Te gustaría que intercambiemos los trabajos?')]


**5. Train–Validation Split (CPU-Friendly Subset)**

In [20]:
split_idx = int(0.8 * len(pairs))

train_pairs = pairs[:split_idx][:40000]
valid_pairs = pairs[split_idx:][:8000]

print("Train pairs:", len(train_pairs))
print("Valid pairs:", len(valid_pairs))


Train pairs: 40000
Valid pairs: 8000


**6. Text Vectorization**

In [21]:
vocab_size = 1000
max_length = 50

train_en = [en for en, _ in train_pairs]
train_es = [es for _, es in train_pairs]

text_vec_en = keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_sequence_length=max_length
)

text_vec_es = keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_sequence_length=max_length
)

text_vec_en.adapt(train_en)
text_vec_es.adapt([f"startofseq {s} endofseq" for s in train_es])

print(text_vec_en.get_vocabulary()[:10])
print(text_vec_es.get_vocabulary()[:10])


['', '[UNK]', np.str_('the'), np.str_('i'), np.str_('to'), np.str_('you'), np.str_('tom'), np.str_('a'), np.str_('is'), np.str_('he')]
['', '[UNK]', np.str_('startofseq'), np.str_('endofseq'), np.str_('de'), np.str_('que'), np.str_('a'), np.str_('no'), np.str_('tom'), np.str_('la')]


**7. Prepare Training and Validation Tensors**

In [22]:
train_en = [en for en, _ in train_pairs]
train_es = [es for _, es in train_pairs]
valid_en = [en for en, _ in valid_pairs]
valid_es = [es for _, es in valid_pairs]

X_train_enc = tf.constant(train_en)
X_valid_enc = tf.constant(valid_en)

X_train_dec = tf.constant([f"startofseq {s}" for s in train_es])
X_valid_dec = tf.constant([f"startofseq {s}" for s in valid_es])

Y_train = text_vec_es([f"{s} endofseq" for s in train_es])
Y_valid = text_vec_es([f"{s} endofseq" for s in valid_es])

print("Target shape:", Y_train.shape)


Target shape: (40000, 50)


**8. Transformer Architecture (Attention Is All You Need)**

In [28]:
import numpy as np
from tensorflow import keras
import tensorflow as tf

embed_size = 128
num_heads = 8
ff_dim = 512
num_layers = 2
dropout_rate = 0.1

class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_len, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.max_len = max_len
        self.embed_dim = embed_dim

        pos = np.arange(max_len)[:, None]
        i = np.arange(embed_dim)[None, :]
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / embed_dim)
        angles = pos * angle_rates

        angles[:, 0::2] = np.sin(angles[:, 0::2])
        angles[:, 1::2] = np.cos(angles[:, 1::2])

        self.pos_encoding = tf.constant(angles[None, ...], dtype=tf.float32)

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]

    def get_config(self):
        config = super().get_config()
        config.update({
            "max_len": self.max_len,
            "embed_dim": self.embed_dim,
        })
        return config

def transformer_encoder(x):
    attn = keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate
    )(x, x)
    x = keras.layers.LayerNormalization()(x + attn)
    ffn = keras.Sequential([
        keras.layers.Dense(ff_dim, activation="relu"),
        keras.layers.Dense(embed_size)
    ])
    return keras.layers.LayerNormalization()(x + ffn(x))

def transformer_decoder(x, enc_out):
    attn1 = keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate
    )(x, x, use_causal_mask=True)
    x = keras.layers.LayerNormalization()(x + attn1)

    attn2 = keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate
    )(x, enc_out)
    x = keras.layers.LayerNormalization()(x + attn2)

    ffn = keras.Sequential([
        keras.layers.Dense(ff_dim, activation="relu"),
        keras.layers.Dense(embed_size)
    ])
    return keras.layers.LayerNormalization()(x + ffn(x))

encoder_inputs = keras.layers.Input(shape=(), dtype=tf.string)
decoder_inputs = keras.layers.Input(shape=(), dtype=tf.string)

enc_tokens = text_vec_en(encoder_inputs)
dec_tokens = text_vec_es(decoder_inputs)

embedding = keras.layers.Embedding(vocab_size, embed_size)

enc_embed = PositionalEncoding(max_length, embed_size)(embedding(enc_tokens))
dec_embed = PositionalEncoding(max_length, embed_size)(embedding(dec_tokens))

x = enc_embed
for _ in range(num_layers):
    x = transformer_encoder(x)

y = dec_embed
for _ in range(num_layers):
    y = transformer_decoder(y, x)

outputs = keras.layers.Dense(vocab_size, activation="softmax")(y)

transformer_model = keras.Model(
    inputs=[encoder_inputs, decoder_inputs],
    outputs=outputs
)

transformer_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

transformer_model.summary()


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_vectorization… │ (None, 50)        │          0 │ input_layer_7[0]… │
│ (TextVectorization) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_vectorization… │ (None, 50)        │          0 │ input_layer_6[0]… │
│ (TextVectorization) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 50, 128)   │    128,000 │ text_vectorizati… │
│ (Embedding)         │                   │            │ text_vectorizati… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_encodin… │ (None, 50, 128)   │          0 │ embedding_1[0][0] │
│ (PositionalEncodin… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 50, 128)   │    527,488 │ positional_encod… │
│ (MultiHeadAttentio… │                   │            │ positional_encod… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_10 (Add)        │ (None, 50, 128)   │          0 │ positional_encod… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 50, 128)   │        256 │ add_10[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_4        │ (None, 50, 128)   │    131,712 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_11 (Add)        │ (None, 50, 128)   │          0 │ layer_normalizat… │
│                     │                   │            │ sequential_4[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 50, 128)   │        256 │ add_11[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 50, 128)   │    527,488 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_12 (Add)        │ (None, 50, 128)   │          0 │ layer_normalizat… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_encodin… │ (None, 50, 128)   │          0 │ embedding_1[1][0] │
│ (PositionalEncodin… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 50, 128)   │        256 │ add_12[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 50, 128)   │    527,488 │ positional_encod

 Total params: 3,951,336 (15.07 MB)

 Trainable params: 3,951,336 (15.07 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# checkpoint_cb = keras.callbacks.ModelCheckpoint(
#     "attention_best.keras",
#     save_best_only=True,
#     monitor="val_loss"
# )

# history = attention_model.fit(
#     (X_train_enc, X_train_dec),
#     Y_train,
#     validation_data=((X_valid_enc, X_valid_dec), Y_valid),
#     epochs=10,
#     batch_size=64,
#     callbacks=[checkpoint_cb]
# )


In [25]:
# history = attention_model.fit(
#     (X_train_enc, X_train_dec),
#     Y_train,
#     validation_data=((X_valid_enc, X_valid_dec), Y_valid),
#     epochs=2,
#     batch_size=64
# )


**9. Training with Early Stopping & Checkpointing**

In [26]:
best_model_cb = keras.callbacks.ModelCheckpoint(
    "transformer_best.keras",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

early_stop_cb = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True,
    verbose=1
)

history = transformer_model.fit(
    (X_train_enc, X_train_dec),
    Y_train,
    validation_data=((X_valid_enc, X_valid_dec), Y_valid),
    epochs=8,
    batch_size=128,
    callbacks=[best_model_cb, early_stop_cb]
)


Epoch 1/8
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8525 - loss: 1.1583
Epoch 1: val_loss improved from inf to 0.65686, saving model to transformer_best.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 2386s 8s/step - accuracy: 0.8525 - loss: 1.1571 - val_accuracy: 0.8839 - val_loss: 0.6569
Epoch 2/8
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8884 - loss: 0.6439
Epoch 2: val_loss improved from 0.65686 to 0.56877, saving model to transformer_best.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 2409s 8s/step - accuracy: 0.8885 - loss: 0.6439 - val_accuracy: 0.8996 - val_loss: 0.5688
Epoch 3/8
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9002 - loss: 0.5544
Epoch 3: val_loss improved from 0.56877 to 0.51546, saving model to transformer_best.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 2379s 8s/step - accuracy: 0.9002 - loss: 0.5543 - val_accuracy: 0.9032 - val_loss: 0.5155
Epoch 4/8
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9044 - loss: 0.5024
Epoch 4: val_loss improved from 0.5154

**10. Load Best Saved Model**

In [29]:
from tensorflow import keras

transformer_model = keras.models.load_model(
    "transformer_best.keras",
    custom_objects={"PositionalEncoding": PositionalEncoding}
)

print("Transformer model loaded successfully")


Transformer model loaded successfully


**11. Inference (Greedy Decoding)**

In [30]:
def translate(sentence):
    translation = ""
    for step in range(max_length):
        X = tf.constant([sentence])
        X_dec = tf.constant([f"startofseq {translation}"])
        probs = transformer_model.predict((X, X_dec), verbose=0)[0, step]
        word_id = np.argmax(probs)
        word = text_vec_es.get_vocabulary()[word_id]
        if word == "endofseq":
            break
        translation += " " + word
    return translation.strip()

print("EN:", "I love sports.")
print("PR:", translate("I love sports."))


EN: I love sports.
PR: [UNK] [UNK]


**12. BLEU Score Evaluation**

In [31]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

nltk.download("punkt")
smooth = SmoothingFunction().method1

def compute_bleu(n=200):
    scores = []
    for i in range(n):
        en, ref = valid_pairs[i]
        pred = translate(en)
        scores.append(
            sentence_bleu(
                [ref.lower().split()],
                pred.lower().split(),
                smoothing_function=smooth
            )
        )
    return np.mean(scores)

print("Average BLEU score:", compute_bleu())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Average BLEU score: 0.027294195107791346


**13. Qualitative Translation Results**

In [32]:
for i in range(5):
    en, ref = valid_pairs[i]
    print("EN:", en)
    print("GT:", ref)
    print("PR:", translate(en))
    print("-" * 50)


EN: We're investigating it.
GT: Lo estamos investigando.
PR: [UNK] [UNK]
--------------------------------------------------
EN: My son is a rebellious teenager.
GT: Mi hijo es un adolescente rebelde.
PR: mi padre se [UNK] en el [UNK]
--------------------------------------------------
EN: I'm tired.
GT: Yo estoy cansada.
PR: estoy [UNK]
--------------------------------------------------
EN: I cooked dinner.
GT: Cociné la cena.
PR: [UNK] [UNK]
--------------------------------------------------
EN: What made you come here?
GT: Qué te trajo aquí?
PR: qué te [UNK]
--------------------------------------------------


In [33]:
from tensorflow.keras.utils import plot_model

plot_model(
    transformer_model,
    to_file="transformer_architecture.png",
    show_shapes=True,
    show_layer_names=True,
    expand_nested=True,
    dpi=150
)

print("Transformer architecture diagram saved as transformer_architecture.png")


Transformer architecture diagram saved as transformer_architecture.png
